In [1]:
import pandas as pd
import numpy as np
from tqdm import *
import re
tqdm.pandas()


/Users/kamakshibansal/opt/anaconda3/lib/python3.7/site-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
def return_nan(tup):
    return([None] if len(tup[0]) == 0 else tup[0])

def get_indexes_match(l1, l2):

    return([return_nan(np.where([x in e for x in l2])) for e in l1])

def merge(df1, df2, left_on, right_on):
    df1.loc[:, 'idx'] = get_indexes_match(df1[left_on].values,
                                          df2[right_on].values)
    
def clean_keywords(keyword):
    return re.sub('[^.,a-zA-Z0-9 \n\.]', '', str(keyword))

def clean_sentences(sentence):
    result1= sentence.replace(', ', ',')
    return result1


def clean_mr_and_remove_stopwords(query, merge_dict):
    result2 = query.split(",")
    resultwords  = [word for word in result2 if merge_dict[word]==0]
    final= ','.join(resultwords)
    final= final.rstrip()
    return final

In [8]:
summ=0
for i in range(1999, 2020):
    File= pd.read_csv("/Users/kamakshibansal/Moorfields/Top_11_Diag.csv")
    File["Diagnosis"]= File["Diagnosis"].str.lower()

    wordFrequency_name = dict.fromkeys(File["Diagnosis"].str.lower().unique().tolist(),1 )
    
    print ("Working on year {} file".format(i))
    FileL = pd.read_csv("//Users/kamakshibansal/Moorfields/Letters/Filtered_Symptoms/File_withsymptoms{}.csv".format(i))
    FileL=FileL.dropna()
    print ("The size of {} file is ".format(i), FileL.shape)

    FileL["LetterBody"]= FileL["LetterBody"].str.lower()
    
    merge(FileL, File, left_on='LetterBody', right_on='Diagnosis')
    
    File["Diagnosis"] = File.values.tolist()
    FileL['Diagnosis'] = FileL.idx.apply(lambda x: [File["Diagnosis"].get(i) for i in x])

    FileL['Diagnosis2'] = FileL.Diagnosis.map(clean_keywords)
    FileL= FileL.drop(["Diagnosis","idx"], axis=1)

    File_Symp= FileL[FileL['Diagnosis2']!="None"]
    File_Symp['count'] = File_Symp.Diagnosis2.apply(lambda x: len(x.split(',')))
    # change the count to 2 in case you want a file with 2 diagnosis
    File_with_1_diagnosis= File_Symp[File_Symp['count']==1]
    
    print ("The size of {} file filtering with top 11 diagnosis is ".format(i), File_Symp.shape)
    print("The size of {} file with one diagnosis as lablel is ".format(i), File_with_1_diagnosis.shape)
    
    a= len(File_with_1_diagnosis)
#     print(a)
    summ= summ + a
#     File_with_1_diagnosis= File_with_1_diagnosis[["final_sentence", 'Diagnosis2']]
    File_with_1_diagnosis= File_with_1_diagnosis[["Symptoms2", 'Diagnosis2']]
    
    # To remove the diagnosis from the symptoms if exist, as if same label is in input, the accuracy will be very high.
    b=File_with_1_diagnosis["Symptoms2"].map(clean_sentences).str.lower()
    b=b.values.tolist()
    k=','.join(b)
    z=k.split(",")

    z1=list(dict.fromkeys(z))
    wordFrequency = dict.fromkeys(z1,0 )
    merge_dict= { **wordFrequency,**wordFrequency_name}

    %timeit -n 1 -r 1 File_with_1_diagnosis['clean_sentence'] = File_with_1_diagnosis.Symptoms2.progress_map(clean_sentences)
    %timeit -n 1 -r 1 File_with_1_diagnosis['Symptoms'] = File_with_1_diagnosis.clean_sentence.progress_map(lambda c: clean_mr_and_remove_stopwords(c, merge_dict))

    File_with_1_diagnosis= File_with_1_diagnosis[["Symptoms", 'Diagnosis2']]
    
    File_with_1_diagnosis.to_csv("/Users/kamakshibansal/Moorfields/Letters/File_with_1_diagnosis11_symp/File_withDiagnosis{}.csv".format(i))
    
print("The size of all files together is, i.e. sum=", summ)    
    

Working on year 1999 file
The size of 1999 file is  (5771, 8)


/Users/kamakshibansal/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 1137/1137 [00:00<00:00, 235840.15it/s]

The size of 1999 file filtering with top 11 diagnosis is  (1279, 10)
The size of 1999 file with one diagnosis as lablel is  (1137, 10)
8.92 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
7.25 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
The size of all files together is, i.e. sum= 1137


In [4]:
66863+66433+70669+81138+ 94125+ 109094+ 118500+  131568+ 128217

866607

In [5]:
print ("The size of the file is: ", File_Symp.shape)

The size of the file is:  (1279, 10)


In [6]:
File_with_1_diagnosis[["LetterBody", 'Diagnosis2']]

KeyError: "['LetterBody'] not in index"

In [ ]:
# print(File_Symp.shape)

In [ ]:
# File_Symp['count'] = File_Symp.Diagnosis2.apply(lambda x: len(x.split(',')))

In [ ]:
# File_Symp[File_Symp['count']==1]

In [ ]:
1293790

In [ ]:
File= pd.read_csv("/Users/kamakshibansal/Moorfields/Top_11_Diag.csv")

In [ ]:
File

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid")


In [ ]:
plt.figure(figsize=(10,5))
sns.barplot(File['Diagnosis'], File['percentage'])
plt.xticks(rotation=90)